In [1]:
import pandas as pd
from openpyxl import load_workbook
from openpyxl.styles import Font, PatternFill, Alignment
from openpyxl.utils import get_column_letter
import datetime
import locale
locale.setlocale(locale.LC_TIME, 'ru_RU.UTF-8')
from io import BytesIO

In [2]:
# Словарь месяцев вручную, чтобы избежать проблем с кодировкой
months_ru = {
    1: "янв", 2: "фев", 3: "мар", 4: "апр", 5: "май", 6: "июн",
    7: "июл", 8: "авг", 9: "сен", 10: "окт", 11: "ноя", 12: "дек"
}

In [47]:
df = pd.read_excel(r"C:\Users\m.olshanskiy\Desktop\122025_Продажи за ноябрь (1).xlsx", sheet_name='рейтинг')

In [48]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 236 entries, 0 to 235
Data columns (total 2 columns):
 #   Column      Non-Null Count  Dtype 
---  ------      --------------  ----- 
 0   Рейтинг     236 non-null    int64 
 1   Застройщик  236 non-null    object
dtypes: int64(1), object(1)
memory usage: 3.8+ KB


In [49]:
df

,Рейтинг,Застройщик
0,1,ПИК
1,2,Самолет
2,3,А101
3,4,ЛСР
4,5,ДСК-1 (ФСК Лидер)
...,...,...
231,232,Бэсткон
232,233,Инвестстройрегион
233,234,Меджиком
234,235,ПСТ


In [50]:
developers = df["Застройщик"].tolist()

In [51]:
df2 = pd.read_excel(r"C:\Users\m.olshanskiy\Desktop\122025_Продажи за ноябрь (1).xlsx", sheet_name='массив')

In [52]:
def format_col(col):
    if isinstance(col, (pd.Timestamp, datetime.datetime)):
        return f"{months_ru[col.month]}.{str(col.year)[-2:]}"
    return col

df2.columns = [format_col(col) for col in df2.columns]


In [53]:
# Фильтруем по каждому региону
def remove_total_if_one_jk(group):
    # Проверяем сколько строк с ЖК (не Total)
    jk_count = group[group["Название ЖК"] != "Total"].shape[0]
    if jk_count <= 1:
        # Удаляем строки с Total
        group = group[group["Название ЖК"] != "Total"]
    return group

In [54]:
# Проверяем результат
print(df2.columns.tolist())

['Застройщик', 'Регион', 'Название ЖК', 'Среднее за 2024 г., шт', 'янв.25', 'фев.25', 'мар.25', 'апр.25', 'май.25', 'июн.25', 'июл.25', 'авг.25', 'сен.25', 'окт.25', 'ноя.25', 'Среднее за 2025 г., шт']


In [55]:
region_order = ['Total', 'Москва', 'Новая Москва', 'Московская область']

In [56]:
# 2. Создаём словарь: застройщик → порядковый номер
developer_numbers = {name: i + 1 for i, name in enumerate(df["Застройщик"].dropna().unique())}

# 3. Добавляем новый столбец с номерами
df2["№"] = df2["Застройщик"].map(developer_numbers)

# 4. Ставим столбец "№" в начало
df2 = df2[["№"] + [col for col in df2.columns if col != "№"]]





In [57]:
df2['Застройщик'] = pd.Categorical(df2['Застройщик'], categories=developers, ordered=True)
df2['Регион'] = pd.Categorical(df2['Регион'], categories=region_order, ordered=True)
df_sorted = df2.sort_values(['Застройщик', 'Регион']).reset_index(drop=True)

# 5. Дублируем шапку перед каждым застройщиком
header = pd.DataFrame([df_sorted.columns], columns=df_sorted.columns)  # создаём строку-шапку
result = pd.DataFrame(columns=df_sorted.columns)

for dev in df_sorted["№"].unique():
    block = df_sorted[df_sorted["№"] == dev]
    result = pd.concat([result, header, block], ignore_index=True)

df_sorted = result

# 5. Заменяем все значения "Total" на "Итого"
# df_sorted = df_sorted.replace("Total", "Итого")

df_sorted = df_sorted.groupby("Регион", group_keys=False).apply(remove_total_if_one_jk)

C:\Users\m.olshanskiy\AppData\Local\Temp\ipykernel_5340\3313678032.py:18: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  df_sorted = df_sorted.groupby("Регион", group_keys=False).apply(remove_total_if_one_jk)


In [14]:
df_sorted = df_cleaned

In [64]:
# --- 2. Сохраняем в Excel ---
output_file = r"C:\Users\m.olshanskiy\Desktop\Продажи отсортированные 08.12-3.xlsx"
df_sorted.to_excel(output_file, index=False)

# --- 3. Открываем для форматирования ---
wb = load_workbook(output_file)
ws = wb.active

# Проверяем, сколько регионов в файле (не считая общего Итого)
all_regions = [
    str(ws[f"C{row}"].value).strip()
    for row in range(2, ws.max_row + 1)
    if ws[f"C{row}"].value not in [None, "", "Итого"]
]

unique_regions = set(all_regions)
single_region_in_file = len(unique_regions) == 1
print("В файле один регион?", single_region_in_file)

region_start = None
region_end = None
current_region = None
rows_to_delete = []

for row in range(2, ws.max_row + 1):  # первая строка — заголовок
    region = ws[f"C{row}"].value  # регион
    jk_name = ws[f"D{row}"].value  # ЖК / Total

    if region is None:
        continue
    if jk_name is None:
        jk_name = ""

    if region != current_region:
        if region_start is not None:

            # Список ЖК в регионе
            jk_list = [
                str(ws[f"D{r}"].value).strip()
                for r in range(region_start, region_end + 1)
            ]
            total_count = sum(1 for name in jk_list if name.lower() == "total")
            real_jk_count = sum(1 for name in jk_list if name.lower() != "total")

            print(f"\nРегион: {current_region}")
            print(f"ЖК: {jk_list}")
            print(f"real={real_jk_count}, total={total_count}")

            # === УСЛОВИЕ №2: если во всём файле один регион → удаляем все Total ===
            if single_region_in_file:
                for r in range(region_start, region_end + 1):
                    if str(ws[f"D{r}"].value).strip().lower() == "total":
                        rows_to_delete.append(r)
                        print(f"Удаляю строку {r} (Total — единственный регион)")
                # Переходим к следующему региону
                current_region = region
                region_start = row
                continue

            # === УСЛОВИЕ №1: если ЖК внутри региона только один → удаляем Total ===
            if real_jk_count == 1 and total_count > 0:
                for r in range(region_start, region_end + 1):
                    if str(ws[f"D{r}"].value).strip().lower() == "total":
                        rows_to_delete.append(r)
                        print(f"Удаляю строку {r} (Total — один ЖК в регионе)")

        current_region = region
        region_start = row

    region_end = row

# Проверка последнего региона — аналогичная логика
jk_list = [
    str(ws[f"D{r}"].value).strip()
    for r in range(region_start, region_end + 1)
]
total_count = sum(1 for name in jk_list if name.lower() == "total")
real_jk_count = sum(1 for name in jk_list if name.lower() != "total")

if single_region_in_file:
    for r in range(region_start, region_end + 1):
        if str(ws[f"D{r}"].value).strip().lower() == "total":
            rows_to_delete.append(r)
else:
    if real_jk_count == 1 and total_count > 0:
        for r in range(region_start, region_end + 1):
            if str(ws[f"D{r}"].value).strip().lower() == "total":
                rows_to_delete.append(r)

# Удаляем строки с конца
for r in sorted(rows_to_delete, reverse=True):
    ws.delete_rows(r)

# Эта часть добавляет шапку для каждого застройщика, а также добавляет серую заливку и жирный шрифт

# Форматы
fill = PatternFill(start_color="D9D9D9", end_color="D9D9D9", fill_type="solid")
bold_font = Font(bold=True)
center_align = Alignment(vertical="center", horizontal="center")

# --- 4. Форматируем строки-шапки ---
for row in ws.iter_rows(min_row=1, max_row=ws.max_row):
    if row[0].value == "№":  # шапка начинается со "№"
        for cell in row:
            cell.font = bold_font
            cell.fill = fill
            cell.alignment = center_align

# --- 5. Объединяем одинаковые подряд ячейки ---
def merge_identical_cells(column_idx):
    start = 2  # пропускаем первую строку
    current_value = ws[f"{get_column_letter(column_idx)}{start}"].value
    for row in range(3, ws.max_row + 2):
        cell_value = ws[f"{get_column_letter(column_idx)}{row}"].value
        if cell_value != current_value:
            if row - start > 1 and current_value is not None:
                ws.merge_cells(start_row=start, start_column=column_idx,
                               end_row=row - 1, end_column=column_idx)
                ws[f"{get_column_letter(column_idx)}{start}"].alignment = center_align
            start = row
            current_value = cell_value

# Объединяем по нужным столбцам
merge_identical_cells(1)  # №
merge_identical_cells(2)  # Застройщик
merge_identical_cells(3)  # Регион
merge_identical_cells(4)  # Название ЖК (если нужно)




# --- 6. Сохраняем итог ---
wb.save(output_file)
print("✅ Готово! Шапки добавлены, выделены цветом, и ячейки объединены.")


В файле один регион? False

Регион: Регион
ЖК: ['Название ЖК']
real=1, total=0

Регион: Total
ЖК: ['None']
real=1, total=0

Регион: Москва
ЖК: ['Total', 'Бусиновский парк', 'Люблинский парк', 'Москворечье', 'Зеленый парк', 'Матвеевский парк', 'Плеханова 11', 'Большая Академическая 85', 'Алтуфьевское 53', 'Холланд Парк', 'Волжский парк', 'Кавказский 51', 'Никольские луга', 'Мичуринский парк', 'Второй Иртышский', 'Руставели 14', 'Амурский Парк', 'Полар', 'Второй Нагатинский', 'Первый Дубровский', 'Митинский лес', 'Барклая 6', 'Сигнальный 16', 'Ютаново', 'Кутузовский квартал', 'Новое Очаково', 'Кольская 8', 'Кронштадтский 9', 'Римского-Корсакова 11', 'Полярная 25', 'Квартал Мит', 'Открытый парк', 'Нарвин', 'Большая Очаковская 2', 'Кронштадтский 14', 'Лосиноостровский парк', 'Онежский вал', 'Красноказарменная 15', 'Перовское 2', 'Вангарден']
real=39, total=1

Регион: Новая Москва
ЖК: ['Total', 'Саларьево Парк', 'Середневский лес', 'Юнино', 'Бунинские луга', 'Бунинская набережная']
real=5, 